### Importación de librerías

In [79]:
from bs4 import BeautifulSoup # Herramienta de web scrapping

import requests # Herramienta para consultar paginas web

import pandas as pd # Herramienta de manipulacion de datos (reemplaza a excel)

import random # Generación de números aleatorios

from datetime import datetime # Herramienta para trabajar con fechas

from lxml import etree

# Herramienta de visualizacion de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Para texto
import re # Herramienta para expresiones regulares
import nltk
from nltk.corpus import stopwords # Herramienta de procesamiento de lenguaje natural
from unidecode import unidecode # Traduce caracateres especiales

In [2]:
sns.set(style="ticks", color_codes=True) # Estilo de gráficos
%matplotlib inline # Visualizar los gráficos dentro del notebook

UsageError: unrecognized arguments: # Visualizar los gráficos dentro del notebook


### REVISAR

Acontinuación vamos a extraer la fecha de un post en Facebook
... Esto lo haremos a modo de ejemplo debido a que extraer todo el contenido que necesitamos para el análisis tomaría mucho tiempo

In [36]:
r = requests.get("https://www.facebook.com/McDonaldsPeru/photos/a.173819762664531/2626129120766904/?type=3&theater")
soup = BeautifulSoup(r.content, 'html.parser')
r.ok

True

In [37]:
html = etree.HTML(str(soup))

In [42]:
# Filtramos segun las etiquetas y los atributos de las mismas para obtener la fecha de publicación
post_date = html.xpath("/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div/div[2]/div/div/div/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[2]/span/span/span[2]/span/a/text()")

In [43]:
print(post_date)

[]


### Importación de datos

In [45]:
df = pd.read_csv('../src/data/test.csv')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


In [50]:
df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


#### Ahora calculemos el sentimiento de estos comentarios
Debemos seguir los siguientes pasos:

1. Obtener un diccionario de palabras positivas y negativas
2. Limpiar las palabras del diccionario y los comentarios
3. Calcular el numero de palabras asociadas a cada sentimiento por comentario
4. Visualizar el resultado

In [154]:
# Cargamos las palabras y su sentimiento
#words_sentiment = pd.read_csv('../src/data/words_sentiment.csv')
possitive_words = pd.read_csv('../src/data/positive words.csv', usecols=[1], names=["word"])
negative_words = pd.read_csv('../src/data/negative words.csv',  encoding = "ISO-8859-1", usecols=[1], names=["word"])

In [155]:
possitive_words['sentiment'] = 'positive'
negative_words['sentiment'] = 'negative'

In [156]:
possitive_words.head(2)

,word,sentiment
0,abound,positive
1,abounds,positive


In [157]:
words_sentiment = pd.concat([possitive_words,negative_words])

In [158]:
# Obtenemos informacion general de los datos
words_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6786 entries, 0 to 4780
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   word       6786 non-null   object
 1   sentiment  6786 non-null   object
dtypes: object(2)
memory usage: 159.0+ KB


In [159]:
# Obtenemos un vistazo de las primeras 5 filas de los datos
words_sentiment.head()

,word,sentiment
0,abound,positive
1,abounds,positive
2,abundance,positive
3,abundant,positive
4,accessable,positive


In [160]:
# Limpiamos las palabras del diccionario
words_sentiment['word'] = words_sentiment['word'].apply(unidecode) # Elimina caracteres especiales
words_sentiment['word'] = words_sentiment['word'].str.lower() # Pone todo en minusculas

In [161]:
# Lista de palabras positivas
pos_words = words_sentiment[words_sentiment['sentiment'] == 'positive']
pos_words = pos_words['word'].values.tolist()

# Lista de palabras negativas
neg_words = words_sentiment[words_sentiment['sentiment'] == 'negative']
neg_words = neg_words['word'].values.tolist()

#### Ahora seleccionaremos un comentario como ejemplo

In [162]:
text = df.loc[20,'tweet']
print(text)

hey guys tommorow is the last day of my exams i'm so happy yay  


In [163]:
# Limpiamos el comentario
text_ = unidecode(text) # Elimina caracateres especiales
text_ = re.sub("[^ a-zA-Z]", "", text_) # Elimina puntuacion y otros
text_ = text_.lower() # Pone todo en minúsculas
print(text_)

hey guys tommorow is the last day of my exams im so happy yay  


In [164]:
tokens = text_.split(" ") # Separa las palabras por espacios -> lista de tokens            
print(tokens)

['hey', 'guys', 'tommorow', 'is', 'the', 'last', 'day', 'of', 'my', 'exams', 'im', 'so', 'happy', 'yay', '', '']


In [166]:
# Obten las palabras stopwords
stopwords_set = set(stopwords.words("english")) 
# Eliminamos las palabras stopwords
clean_words = [t for t in tokens if not t in stopwords_set]
print(clean_words)

['hey', 'guys', 'tommorow', 'last', 'day', 'exams', 'im', 'happy', 'yay', '', '']


In [167]:
# Conteo de palabras positivas y negativas
pos_word_count = 0
neg_word_count = 0

for word in clean_words:
    if word in pos_words:
        pos_word_count += 1
    if word in neg_words:
        neg_word_count += 1

print('# de palabras positivas: ', pos_word_count)
print('# de palabras negativas: ', neg_word_count)

# de palabras positivas:  2
# de palabras negativas:  0
